In [2]:
import keras

In [3]:
from keras.layers import Conv2D, Dense, MaxPooling2D

In [4]:
def encode(seq):
    n = len(seq)
    l = [[0 for i in range(n)] for j in range(n)]
    for i in range(n):
        for j in range(n):
            if((seq[i] == 'A' and seq[j] == 'U') or (seq[j] == 'A' and seq[i] == 'U')):
                l[i][j] = 2
            elif((seq[i] == 'G' and seq[j] == 'C') or (seq[j] == 'G' and seq[i] == 'C')):
                l[i][j] = 3
            elif((seq[i] == 'G' and seq[j] == 'U') or (seq[j] == 'U' and seq[i] == 'G')):
                l[i][j] = 0.8
            else:
                l[i][j] = 0
    return l

In [5]:
import numpy as np
import math

def Gaussian(x):
    return math.exp(-0.5*(x*x))

def matrix_calc(data, sliding_param):
    mat = np.zeros([len(data)+sliding_param,len(data)])
    for i in range(len(data[0])):
        for j in range(len(data[0])):
            coefficient = 0
            for add in range(30):
                if i - add >= 0 and j + add <len(data):
                    score = data[i-add][j+add]
                    if score == 0:
                        break
                    else:
                        coefficient = coefficient + score * Gaussian(add)
                else:
                    break
            if coefficient > 0:
                for add in range(1,30):
                    if i + add < len(data) and j - add >= 0:
                        score = data[i+add][j-add]
                        if score == 0:
                            break
                        else:
                            coefficient = coefficient + score * Gaussian(add)
                    else:
                        break
            mat[[i+int(sliding_param/2)],[j]] = coefficient
    return mat

def slidingWindow(numpy_M, sliding_param, n):
    zeros = np.zeros((sliding_param, n), dtype=int)
    numpy_M = np.append(numpy_M, zeros, axis=0)
    sliding_mats = [numpy_M[i: i + sliding_param, :].tolist() for i in range(n)]
    return sliding_mats

In [6]:
print(encode("ACUGGGAUAAAA"))
x = slidingWindow(matrix_calc(encode("ACUGGGAUAAAA"), 3), 3, 12)

[[0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0], [0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2], [0, 3, 0.8, 0, 0, 0, 0, 0.8, 0, 0, 0, 0], [0, 3, 0.8, 0, 0, 0, 0, 0.8, 0, 0, 0, 0], [0, 3, 0.8, 0, 0, 0, 0, 0.8, 0, 0, 0, 0], [0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2], [0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0], [0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0], [0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0], [0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0]]


In [7]:
def extractFromText(file):
    f = open(file, "r")
    lines = f.readlines()
    seq = lines[3].strip()
    dbn = lines[4]
    dots = [[0 for i in range(3)] for j in range(len(dbn))]
    for i in range(len(dbn)):
        if(dbn[i] == '('):
            dots[i][0] = 1
            dots[i][1] = 0
            dots[i][2] = 0
        elif(dbn[i] == ')'):
            dots[i][0] = 0
            dots[i][1] = 1
            dots[i][2] = 0
        else:
            dots[i][0] = 0
            dots[i][1] = 0
            dots[i][2] = 1
    return seq, np.array(dots)
    
            
def scale(im, nR, nC):
    nR0 = len(im) 
    nC0 = len(im[0]) 
    arr =  [[ im[int(nR0 * r / nR)][int(nC0 * c / nC)]  
             for c in range(nC)] for r in range(nR)]
    return np.array(arr)

    

In [8]:
x = extractFromText(r"C:\Users\sidsr\Downloads\dbnFiles\dbnFiles\bpRNA_CRW_4300.dbn")
print(x)
for i in x[1]:
    print(i)

('GGGUGAGUACAAAGGAAGCGUCCGUUCCUUGAGAACUCAACAGCGUGCCAAAAGUCAACGCCAGAUAUGUUGAUACCCCGACCUGAUCGGAUCUCCGUUCGGGUUGAGGUUCCUUUGAAGUAACACAACAGCGAGGACGCUGUGAACGGUCGGAUUAUUCCUCCGACUGUUCCGCUCUCGUGGUGUCACCCGAUUACGGGUAUACAUUCACGGAGAGUUUGAUCCUGGCUCAGGACGAACGCUGGCGGCGUGCUUAACACAUGCAAGUCGAACGAUGAACCACUUCGGUGGGGAUUAGUGGCGAACGGGUGAGUAACACGUGGGCAAUCUGCCCUUCACUCUGGGACAAGCCCUGGAAACGGGGUCUAAUACCGGAUACUGACCCUCGCAGGCAUCUGCGAGGUUCGAAAGCUCCGGCGGUGAAGGAUGAGCCCGCGGCCUAUCAGCUUGUUGGUGAGGUAAUGGCUCACCAAGGCGACGACGGGUAGCCGGCCUGAGAGGGCGACCGGCCACACUGGGACUGAGACACGGCCCAGACUCCUACGGGAGGCAGCAGUGGGGAAUAUUGCACAAUGGGCGAAAGCCUGAUGCAGCGACGCCGCGUGAGGGAUGACGGCCUUCGGGUUGUAAACCUCUUUCAGCAGGGAAGAAGCGAAAGUGACGGUACCUGCAGAAGAAGCGCCGGCUAACUACGUGCCAGCAGCCGCGGUAAUACGUAGGGCGCAAGCGUUGUCCGGAAUUAUUGGGCGUAAAGAGCUCGUAGGCGGCUUGUCACGUCGGUUGUGAAAGCCCGGGGCUUAACCCCGGGUCUGCAGUCGAUACGGGCAGGCUAGAGUUCGGUAGGGGAGAUCGGAAUUCCUGGUGUAGCGGUGAAAUGCGCAGAUAUCAGGAGGAACACCGGUGGCGAAGGCGGAUCUCUGGGCCGAUACUGACGCUGAGGAGCGAAAGCGUGGGGAGCGAACAGGAUUAGAUACCCUGGUAGUCCACGCCGUAAACGG

In [23]:
training_data = np.zeros(shape=(500000, 19, 60))
training_labels = np.zeros(shape=(500000, 1, 3))

import os
path = r"C:\Users\sidsr\Downloads\dbnFiles\dbnFiles"
files = os.listdir(path)

train_counter = 0
for i in range(100000, 102000):
    x = extractFromText(path+"\\"+files[i])
    seq = x[0]
    windows = slidingWindow(matrix_calc(encode(seq), 19), 19, len(seq))
    for j in range(len(windows)):
        training_data[train_counter] = scale(windows[j], 19, 60)
        training_labels[train_counter] = x[1][j]
        train_counter+=1



In [24]:
print(train_counter)

380448


In [26]:
training_data = np.resize(training_data, (train_counter, 19, 60))
training_labels = np.resize(training_labels, (train_counter, 1, 3))

In [27]:
np.save("train_data", training_data)
np.save("train_label", training_labels)

In [16]:
training_data = np.load("train_data.npy")
training_labels = np.load("train_label.npy")

In [17]:
model = keras.Sequential()
model.add(Conv2D(filters = 16, kernel_size = 3, input_shape=(19,  60, 1), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size = (3, 3), padding='same'))
model.add(Conv2D(filters = 16, kernel_size = 3, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size = (3, 3), padding='same'))
model.add(Conv2D(filters = 16, kernel_size = 3, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size = (3, 3), padding='same'))

model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 19, 60, 16)        160       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 7, 20, 16)        0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 7, 20, 16)         2320      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 3, 7, 16)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 7, 16)          2320      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 1, 3, 16)        

In [19]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy', 'loss'], optimizer='adam')

In [20]:
batch_size = 32
epochs = 100
print(training_data.shape)
print(training_labels.shape)
model.fit(training_data, training_labels, batch_size=batch_size, epochs=epochs, validation_split=0.3,  verbose=1)

(380448, 1, 3)


NameError: name 'training_labels' is not defined